In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import cv2
import os
import yaml
from ultralytics import YOLO
from PIL import Image, ImageTk
import numpy as np

# Load the trained YOLO model
model = YOLO(r'best.pt')  # Adjust path to your model weights

# Load class names from the dataset.yaml
yaml_path = r'dataset.yaml'  # Path to your YAML file
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)
class_names = data['names']  # This assumes your .yaml file has a 'names' field listing the classes

# Path to folder with unlabeled images
unlabeled_images_folder = r'image_to_test'
output_labels_folder = r'image_to_test_label'
os.makedirs(output_labels_folder, exist_ok=True)

# Set a lower confidence threshold (e.g., 0.5)
confidence_threshold = 0.5

class BoundingBoxEditor:
    def __init__(self, root):
        self.root = root
        self.root.title("Bounding Box Editor")

        # Create a frame for the image list and the main content
        self.main_frame = tk.Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)

        # Create a Listbox for the image list with a scrollbar
        self.image_list_frame = tk.Frame(self.main_frame, width=200)
        self.image_list_frame.pack(side=tk.LEFT, fill=tk.Y)

        self.image_listbox = tk.Listbox(self.image_list_frame)
        self.image_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar = tk.Scrollbar(self.image_list_frame, orient="vertical", command=self.image_listbox.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.image_listbox.configure(yscrollcommand=self.scrollbar.set)

        # Note: We do NOT bind mouse scrolling to the image listbox, so scrolling will not trigger image changes.
        # This allows us to scroll the image list without switching images.

        # Create a frame for the canvas and the info panel
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Create a canvas for displaying the image
        self.canvas = tk.Canvas(self.content_frame, width=500, height=720)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Bind mouse scroll events to the canvas for scrolling through images
        self.canvas.bind("<MouseWheel>", self.on_mouse_wheel)  # For Windows and MacOS
        self.canvas.bind("<Button-4>", self.on_mouse_wheel)  # For Linux
        self.canvas.bind("<Button-5>", self.on_mouse_wheel)  # For Linux

        # Create a frame for displaying bounding box information
        self.info_frame = tk.Frame(self.content_frame, width=300)
        self.info_frame.pack(side=tk.LEFT, fill=tk.Y)

        self.info_label = tk.Label(self.info_frame, text="Bounding Box Info", font=("Arial", 14))
        self.info_label.pack(pady=10)

        self.image_name_label = tk.Label(self.info_frame, text="", font=("Arial", 12))
        self.image_name_label.pack(pady=5)

        self.bbox_info_frame = tk.Frame(self.info_frame)
        self.bbox_info_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        # Create a frame for displaying class options
        self.class_frame = tk.Frame(self.content_frame, width=200)
        self.class_frame.pack(side=tk.RIGHT, fill=tk.Y)

        self.class_label = tk.Label(self.class_frame, text="Classes", font=("Arial", 14))
        self.class_label.pack(pady=10)

        self.class_listbox = tk.Listbox(self.class_frame)
        self.class_listbox.pack(pady=10, fill=tk.BOTH, expand=True)
        for class_name in class_names:
            self.class_listbox.insert(tk.END, class_name)

        self.clear_selection_button = tk.Button(self.class_frame, text="Clear Selection", command=self.clear_class_selection)
        self.clear_selection_button.pack(pady=10)

        # Add a Paste All button for pasting all copied bounding boxes
        self.paste_all_button = tk.Button(self.class_frame, text="Paste All", command=self.paste_all_bboxes)
        self.paste_all_button.pack(pady=10)

        self.delete_image_button = tk.Button(self.class_frame, text="Delete Image", command=self.delete_image)
        self.delete_image_button.pack(pady=10)

        # Add a frame for copied bounding boxes
        self.copy_frame = tk.Frame(self.class_frame)
        self.copy_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        # Bind mouse scroll events to the copied bounding boxes frame
        self.copy_frame.bind("<MouseWheel>", self.on_mouse_wheel)  # For Windows and MacOS
        self.copy_frame.bind("<Button-4>", self.on_mouse_wheel)  # For Linux
        self.copy_frame.bind("<Button-5>", self.on_mouse_wheel)  # For Linux

        self.copied_bbox_list = []  # List to store all copied bounding boxes
        self.update_copied_bbox_display()

        self.image = None
        self.image_path = None
        self.bboxes = []
        self.current_bbox = None
        self.rect = None
        self.image_files = []
        self.current_image_index = -1
        self.folder_path = None
        self.selected_class_index = None  # Store the selected class index


        self.copied_bbox = None
        
        self.load_folder_button = tk.Button(root, text="Load Folder", command=self.load_folder)
        self.load_folder_button.pack(side=tk.LEFT)

        self.save_button = tk.Button(root, text="Save Labels", command=self.save_labels)
        self.save_button.pack(side=tk.LEFT)

        self.canvas.bind("<Button-1>", self.on_click)
        self.canvas.bind("<B1-Motion>", self.on_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_release)
        self.root.bind("s", lambda event: self.save_labels())
        self.root.bind("<Escape>", lambda event: self.clear_class_selection())
        self.root.bind("<Down>", self.next_image)
        self.root.bind("<Up>", self.previous_image)

        # Unbind the default behavior of the Listbox for the up and down arrow keys
        self.class_listbox.bind("<Down>", lambda e: "break")
        self.class_listbox.bind("<Up>", lambda e: "break")

        # self.image_listbox.bind("<MouseWheel>", self.on_mouse_wheel)  # For Windows and MacOS
        # self.image_listbox.bind("<Button-4>", self.on_mouse_wheel)  # For Linux
        # self.image_listbox.bind("<Button-5>", self.on_mouse_wheel)  # For Linux

        self.image_listbox.bind("<<ListboxSelect>>", self.on_image_select)

        # Bind class selection to store the selected index
        self.class_listbox.bind("<<ListboxSelect>>", self.on_class_select)

    def update_copied_bbox_display(self):
        """Update the display of copied bounding boxes."""
        # Clear the existing copied frame
        for widget in self.copy_frame.winfo_children():
            widget.destroy()

        # Display each copied bounding box and its class
        if not self.copied_bbox_list:
            copied_label = tk.Label(self.copy_frame, text="Copied Bounding Boxes: None", font=("Arial", 12))
            copied_label.pack(pady=10)
        else:
            for bbox in self.copied_bbox_list:
                x, y, w, h, class_id = bbox
                copied_label = tk.Label(self.copy_frame, text=f"Class {class_names[class_id]}, ({x}, {y}), ({w}, {h})", font=("Arial", 12))
                copied_label.pack(pady=5)
                
    def load_folder(self):
        self.folder_path = unlabeled_images_folder
        if not self.folder_path:
            return

        self.image_listbox.delete(0, tk.END)
        self.image_files = [f for f in os.listdir(self.folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

        # Check if output_labels_folder is empty
        if not os.listdir(output_labels_folder):
            print("The output_labels_folder is empty.")

        for image_file in self.image_files:
            image_path = os.path.join(self.folder_path, image_file)
            label_filename = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(output_labels_folder, label_filename)

            # Check if the label file exists, if not, run inference
            if not os.path.exists(label_path):
                self.run_inference_and_save_labels(image_path, label_path)

            self.image_listbox.insert(tk.END, image_file)
            

    def run_inference_and_save_labels(self, image_path, label_path):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = model(image_path, conf=confidence_threshold)
        bboxes = []

        for box in results[0].boxes:
            class_id = int(box.cls[0])  # Get class ID
            if class_id >= len(class_names):
                continue  # Skip if class_id is out of range
            x_center, y_center, width, height = box.xywhn[0].cpu().numpy()  # Normalized coordinates
            img_height, img_width = image.shape[:2]
            x_center_abs = x_center * img_width
            y_center_abs = y_center * img_height
            width_abs = width * img_width
            height_abs = height * img_height
            x_min = int(x_center_abs - width_abs / 2)
            y_min = int(y_center_abs - height_abs / 2)
            bboxes.append((x_min, y_min, int(width_abs), int(height_abs), class_id))

        with open(label_path, 'w') as label_file:
            for bbox in bboxes:
                x, y, w, h, class_id = bbox
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                width = w / img_width
                height = h / img_height
                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    def on_image_select(self, event):
        selection = event.widget.curselection()
        if selection:
            index = selection[0]
            image_path = os.path.join(self.folder_path, self.image_files[index])
            self.load_image(image_path)

    def on_class_select(self, event):
        selection = event.widget.curselection()
        if selection:
            self.selected_class_index = selection[0]

    def load_image(self, image_path=None):
        """Load a new image and its associated bounding boxes."""
        if image_path:
            self.image_path = image_path
            self.current_image_index = self.image_files.index(os.path.basename(image_path))
        else:
            self.image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
            if not self.image_path:
                return
            self.current_image_index = self.image_files.index(os.path.basename(self.image_path))
    
        self.image = cv2.imread(self.image_path)
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.image = cv2.resize(self.image, (500, 720))  # Resize to 720p
        self.display_image()
    
        # Load existing bounding boxes if available
        label_filename = os.path.splitext(os.path.basename(self.image_path))[0] + '.txt'
        label_path = os.path.join(output_labels_folder, label_filename)
        self.bboxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as label_file:
                for line in label_file:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    img_height, img_width = self.image.shape[:2]
                    x_center_abs = x_center * img_width
                    y_center_abs = y_center * img_height
                    width_abs = width * img_width
                    height_abs = height * img_height
                    x_min = int(x_center_abs - width_abs / 2)
                    y_min = int(y_center_abs - height_abs / 2)
                    self.bboxes.append((x_min, y_min, int(width_abs), int(height_abs), int(class_id)))
    
        self.display_bboxes()

        for i in range(self.image_listbox.size()):
            self.image_listbox.itemconfig(i, bg="white")
        self.image_listbox.itemconfig(self.current_image_index, bg="lightblue")
    
        # Update the image name label
        self.image_name_label.config(text=os.path.basename(self.image_path))
    
        # Reapply the selected class index
        if self.selected_class_index is not None:
            self.class_listbox.selection_set(self.selected_class_index)
            
            # Highlight the selected image name
            for i in range(self.image_listbox.size()):
                self.image_listbox.itemconfig(i, bg="white")
            self.image_listbox.itemconfig(self.current_image_index, bg="lightblue")
    
            # Update the image name label
            self.image_name_label.config(text=os.path.basename(self.image_path))
    
            # Reapply the selected class index
            if self.selected_class_index is not None:
                self.class_listbox.selection_set(self.selected_class_index)

    def display_image(self):
        self.canvas.delete("all")
        self.tk_image = ImageTk.PhotoImage(image=Image.fromarray(self.image))
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

    def display_bboxes(self):
        """Display bounding boxes on the canvas."""
        self.canvas.delete("bbox")
        for widget in self.bbox_info_frame.winfo_children():
            widget.destroy()

        for i, bbox in enumerate(self.bboxes):
            x, y, w, h, class_id = bbox
            self.canvas.create_rectangle(x, y, x + w, y + h, outline="red", width=2, tags="bbox")
            self.canvas.create_text(x, y - 10, text=class_names[class_id], fill="red", anchor=tk.NW, tags="bbox")

            bbox_info = tk.Frame(self.bbox_info_frame)
            bbox_info.pack(fill=tk.X, pady=2)

            bbox_label = tk.Label(bbox_info, text=f"Class: {class_names[class_id]}, Position: ({x}, {y}), Size: ({w}, {h})")
            bbox_label.pack(side=tk.LEFT)

            # Add a Copy button next to each bounding box
            copy_button = tk.Button(bbox_info, text="Copy", command=lambda bbox=bbox: self.copy_bbox(bbox))
            copy_button.pack(side=tk.RIGHT)

            delete_button = tk.Button(bbox_info, text="Delete", command=lambda i=i: self.delete_bbox(i))
            delete_button.pack(side=tk.RIGHT)

    def copy_bbox(self, bbox):
        """Copy the selected bounding box and add it to the copied list."""
        self.copied_bbox_list.append(bbox)
        self.update_copied_bbox_display()

    def paste_all_bboxes(self):
        """Paste all copied bounding boxes into the current image."""
        if self.copied_bbox_list:
            for bbox in self.copied_bbox_list:
                self.bboxes.append(bbox)
            self.display_bboxes()
            print(f"Pasted {len(self.copied_bbox_list)} bounding boxes.")
        else:
            messagebox.showinfo("Info", "No bounding boxes copied to paste.")


    
    def on_click(self, event):
        self.current_bbox = [event.x, event.y, event.x, event.y]
        self.rect = self.canvas.create_rectangle(event.x, event.y, event.x, event.y, outline="blue", width=2, tags="bbox")

    def on_drag(self, event):
        if self.current_bbox is not None:
            self.current_bbox[2] = event.x
            self.current_bbox[3] = event.y
            self.canvas.coords(self.rect, *self.current_bbox)

    def on_release(self, event):
        if self.current_bbox is not None:
            x1, y1, x2, y2 = self.current_bbox
            selected_class_index = self.class_listbox.curselection()
            if selected_class_index:
                class_id = selected_class_index[0]
            else:
                class_id = 0  # Default class ID if none selected
            self.bboxes.append((x1, y1, x2 - x1, y2 - y1, class_id))
            self.current_bbox = None
            self.rect = None
            self.display_bboxes()

    def delete_bbox(self, index):
        del self.bboxes[index]
        self.display_bboxes()

    def save_labels(self):
        if not self.image_path:
            return

        label_filename = os.path.splitext(os.path.basename(self.image_path))[0] + '.txt'
        label_path = os.path.join(output_labels_folder, label_filename)

        with open(label_path, 'w') as label_file:
            for bbox in self.bboxes:
                x, y, w, h, class_id = bbox
                img_height, img_width = self.image.shape[:2]
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                width = w / img_width
                height = h / img_height
                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

        print(f"Saved labels for {self.image_path}")

    def next_image(self, event):
        """Move to the next image in the folder."""
        self.root.focus_set()  # Ensure focus is set to the root window
        if self.current_image_index < len(self.image_files) - 1:
            self.current_image_index += 1
            self.load_image(os.path.join(self.folder_path, self.image_files[self.current_image_index]))

    def previous_image(self, event):
        """Move to the previous image in the folder."""
        self.root.focus_set()  # Ensure focus is set to the root window
        if self.current_image_index > 0:
            self.current_image_index -= 1
            self.load_image(os.path.join(self.folder_path, self.image_files[self.current_image_index]))


    def clear_class_selection(self):
        """Clear the selected class and reset copied bounding boxes."""
        self.class_listbox.selection_clear(0, tk.END)
        self.selected_class_index = None  # Clear the stored class index
        self.copied_bbox_list = []  # Reset the copied bounding boxes
        self.update_copied_bbox_display()
        self.root.focus_set()  # Ensure focus is set to the root window



    def delete_image(self):
        if self.current_image_index == -1:
            messagebox.showwarning("Warning", "No image selected to delete.")
            return
    
        image_path = os.path.join(self.folder_path, self.image_files[self.current_image_index])
        label_filename = os.path.splitext(self.image_files[self.current_image_index])[0] + '.txt'
        label_path = os.path.join(output_labels_folder, label_filename)
    
        # Confirm deletion
        if not messagebox.askyesno("Confirm Delete", f"Are you sure you want to delete {self.image_files[self.current_image_index]} and its label?"):
            return
    
        # Delete the image and label file
        try:
            os.remove(image_path)
            if os.path.exists(label_path):
                os.remove(label_path)
        except Exception as e:
            messagebox.showerror("Error", f"Error deleting files: {e}")
            return
    
        # Remove from the list and update the display
        del self.image_files[self.current_image_index]
    
        # Remove the corresponding label
        self.image_listbox.delete(self.current_image_index)
    
        # Clear the display canvas and reset bbox info
        self.canvas.delete("all")
        self.image_name_label.config(text="")
        self.bbox_info_frame.destroy()
        self.bbox_info_frame = tk.Frame(self.info_frame)
        self.bbox_info_frame.pack(pady=10, fill=tk.BOTH, expand=True)
    
        # If there are still images left, load the next or previous image
        if self.image_files:
            self.current_image_index = min(self.current_image_index, len(self.image_files) - 1)
            self.load_image(os.path.join(self.folder_path, self.image_files[self.current_image_index]))
        else:
            # If no images are left, reset the state
            self.current_image_index = -1

    def on_mouse_wheel(self, event):
        """Handle mouse wheel event for scrolling through images."""
        if event.delta:  # For Windows and MacOS
            if event.delta > 0:
                self.previous_image(event)  # Scroll up for previous image
            else:
                self.next_image(event)  # Scroll down for next image
        elif event.num == 5:  # For Linux (scroll down)
            self.next_image(event)
        elif event.num == 4:  # For Linux (scroll up)
            self.previous_image(event)

if __name__ == "__main__":
    root = tk.Tk()
    editor = BoundingBoxEditor(root)
    editor.load_folder()  # Automatically load the folder on startup
    root.mainloop()

Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_640.jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_88 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_87 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_87 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_84 (4).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_804.jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_801.jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_80 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_79 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_75 (3).jpg
Saved labels for C:\Users\lewka\deep learning\MonsterHNow\image_to_test\frame_748.jpg
Saved labels for C:\Users\lewka\d